# Курс Введение в обработку естественного языка
# Задание к уроку 9. Языковое моделирование
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор

# Решение
Возмьем текст с сонетами Шекспира на английском языке и обучим генератор.

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_to_file = '/content/drive/My Drive/Colab Notebooks/115_NLP/less_09/hw_09/sonnets.txt'

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 95734 characters


In [ ]:
print(text[:500])

FROM fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou, contracted to thine own bright eyes,
Feed'st thy light'st flame with self-substantial fuel,
Making a famine where abundance lies,
Thyself thy foe, to thy sweet self too cruel.
Thou that art now the world's fresh ornament
And only herald to the gaudy spring,
Within thine own bud buriest thy content
And, tender c


In [ ]:
text = text + text

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

61 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int, text, len(text_as_int), len(text)

(array([16, 27, 25, ..., 56, 39,  7]),
 "FROM fairest creatures we desire increase,\r\nThat thereby beauty's rose might never die,\r\nBut as the riper should by time decease,\r\nHis tender heir might bear his memory:\r\nBut thou, contracted to thine own bright eyes,\r\nFeed'st thy light'st flame with self-substantial fuel,\r\nMaking a famine where abundance lies,\r\nThyself thy foe, to thy sweet self too cruel.\r\nThou that art now the world's fresh ornament\r\nAnd only herald to the gaudy spring,\r\nWithin thine own bud buriest thy content\r\nAnd, tender churl, makest waste in niggarding.\r\nPity the world, or else this glutton be,\r\nTo eat the world's due, by the grave and thee.\r\nWhen forty winters shall beseige thy brow,\r\nAnd dig deep trenches in thy beauty's field,\r\nThy youth's proud livery, so gazed on now,\r\nWill be a tatter'd weed, of small worth held:\r\nThen being ask'd where all thy beauty lies,\r\nWhere all the treasure of thy lusty days,\r\nTo say, within thine own 

### train and target

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
R
O
M
 


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

"FROM fairest creatures we desire increase,\r\nThat thereby beauty's rose might never die,\r\nBut as the r"
'iper should by time decease,\r\nHis tender heir might bear his memory:\r\nBut thou, contracted to thine o'
"wn bright eyes,\r\nFeed'st thy light'st flame with self-substantial fuel,\r\nMaking a famine where abunda"
"nce lies,\r\nThyself thy foe, to thy sweet self too cruel.\r\nThou that art now the world's fresh ornamen"
't\r\nAnd only herald to the gaudy spring,\r\nWithin thine own bud buriest thy content\r\nAnd, tender churl,'


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "FROM fairest creatures we desire increase,\r\nThat thereby beauty's rose might never die,\r\nBut as the "
Target data: "ROM fairest creatures we desire increase,\r\nThat thereby beauty's rose might never die,\r\nBut as the r"


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 61) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           7808      
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4722688   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 61)            62525     
Total params: 29,971,133
Trainable params: 29,971,133
Non-trainable params: 0
____________________________________________

In [ ]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 61), dtype=float32, numpy=
array([[-5.3370391e-06, -1.2063516e-05, -6.1564915e-06, ...,
         2.2822492e-06,  6.1387236e-06,  5.8362657e-06],
       [-2.6163476e-05, -4.0847939e-05, -1.7064767e-06, ...,
         8.6626878e-06,  2.5827445e-05,  8.1141116e-06],
       [-7.1164737e-05, -9.5452742e-05,  1.2575823e-05, ...,
         2.0050396e-05,  6.4640742e-05,  9.2401242e-06],
       ...,
       [-8.1966625e-04,  2.4746843e-03, -2.1967101e-03, ...,
         1.8059713e-04,  6.6901365e-04,  3.3052024e-04],
       [-8.3030597e-04,  2.5419421e-03, -2.1419697e-03, ...,
         5.8236415e-05,  6.4508751e-04,  2.8442143e-04],
       [-8.4390206e-04,  2.5836581e-03, -2.1071909e-03, ...,
        -8.6228421e-05,  6.4373121e-04,  2.4319632e-04]], dtype=float32)>

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "t thou bear'st love to any,\r\nWho for thyself art so unprovident.\r\nGrant, if thou wilt, thou art belo"

Next Char Predictions: 
 "dglkJTEF :cG,uo\rk.\r,uy,Ksc,KCty?qoHh.DOv?n;yOkClXnjwqWUppGgBysIIhNe-ql?PFJprqf.:hvNuuhwpK,qsDaf',j \r"


## Train the model

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 61)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.110606


In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

### Execute the training

In [ ]:
EPOCHS = 30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
29/29 [==============================] - 15s 338ms/step - loss: 3.4323
Epoch 2/30
29/29 [==============================] - 10s 346ms/step - loss: 3.1469
Epoch 3/30
29/29 [==============================] - 10s 350ms/step - loss: 3.1411
Epoch 4/30
29/29 [==============================] - 11s 358ms/step - loss: 3.1300
Epoch 5/30
29/29 [==============================] - 11s 367ms/step - loss: 2.9710
Epoch 6/30
29/29 [==============================] - 11s 373ms/step - loss: 2.6309
Epoch 7/30
29/29 [==============================] - 11s 365ms/step - loss: 2.4235
Epoch 8/30
29/29 [==============================] - 10s 349ms/step - loss: 2.2218
Epoch 9/30
29/29 [==============================] - 11s 357ms/step - loss: 2.0911
Epoch 10/30
29/29 [==============================] - 11s 361ms/step - loss: 1.9872
Epoch 11/30
29/29 [==============================] - 11s 365ms/step - loss: 1.8937
Epoch 12/30
29/29 [==============================] - 11s 371ms/step - loss: 1.8003
Epoch 13/30
2

## Generate text

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            7808      
_________________________________________________________________
lstm_4 (LSTM)                (1, None, 1024)           4722688   
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 61)             62525     
Total params: 29,971,133
Trainable params: 29,971,133
Non-trainable params: 0
__________________________________________

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
text_ = generate_text(model, start_string=u"Summertime and your ")
print(text_)

Summertime and your frowing,
And his bramed from my hearles is a look,
Then no come such than the hust the did reck.
Of that be words a seal love will wat to my sore and therein shows,
If it the beauty of thy brain and still,
Spends the ownament with a woman was still
That thou art as our mades to the more,
That then thou wert than that which that thou shouldst thou thrive away,
Thou art to me, to the were buried thing,
That this that which foll with the gainter of the wrong;
And thence as thy fingers, sh
